In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE90713"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE90713"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE90713.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE90713.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE90713.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from human metastatic adrenocortical carcinoma"
!Series_summary	"CXCR4 expression by metastatic adrenocortical carcinoma is heterogeneous among patients and among lesions"
!Series_summary	"We used microarrays for 57 ACC metastases from 42 patients to evaluate gene expression in different lesions from same patients and over time, focusing on CXCR4 expression and other genes correlating with CXCR4 expression"
!Series_overall_design	"57 ACC metastases from 42 patients were used for RNA extraction and hybridization on Affymetrix microarrays. We sought to obtain data on CXCR4 expression by ACC metastases. Multiple lesion samples were aquired for 9 of the patients, labeled a thru i. Single samples were aquired from the other subjects."
Sample Characteristics Dictionary:
{0: ['tissue: adrenocortical carcinoma', 'tissue: normal adrenal'], 1: ['study: 426', 'study: 920'], 2: ['condition: tumor', 'condition: normal'], 3: ['acc_num: 1', 'acc_

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# Based on the background information, this dataset contains microarray data from ACC metastases
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait data: condition (tumor vs normal)
trait_row = 2  # From the sample characteristics dictionary key 2: ['condition: tumor', 'condition: normal']

# Function to convert trait values
def convert_trait(value):
    if isinstance(value, str):
        value = value.strip().lower()
        if "tumor" in value:
            return 1
        elif "normal" in value:
            return 0
    return None

# 2.2 Age data: Not available in the sample characteristics
age_row = None

def convert_age(value):
    return None  # Not used but defined for completeness

# 2.3 Gender data: Not available in the sample characteristics
gender_row = None

def convert_gender(value):
    return None  # Not used but defined for completeness

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM2411058': [1.0], 'GSM2411059': [1.0], 'GSM2411060': [1.0], 'GSM2411061': [1.0], 'GSM2411062': [1.0], 'GSM2411063': [1.0], 'GSM2411064': [1.0], 'GSM2411065': [1.0], 'GSM2411066': [1.0], 'GSM2411067': [1.0], 'GSM2411068': [1.0], 'GSM2411069': [1.0], 'GSM2411070': [1.0], 'GSM2411071': [1.0], 'GSM2411072': [1.0], 'GSM2411073': [1.0], 'GSM2411074': [1.0], 'GSM2411075': [1.0], 'GSM2411076': [1.0], 'GSM2411077': [1.0], 'GSM2411078': [1.0], 'GSM2411079': [1.0], 'GSM2411080': [1.0], 'GSM2411081': [1.0], 'GSM2411082': [1.0], 'GSM2411083': [1.0], 'GSM2411084': [1.0], 'GSM2411085': [1.0], 'GSM2411086': [1.0], 'GSM2411087': [0.0], 'GSM2411088': [0.0], 'GSM2411089': [0.0], 'GSM2411090': [0.0], 'GSM2411091': [0.0], 'GSM2411092': [1.0], 'GSM2411093': [1.0], 'GSM2411094': [1.0], 'GSM2411095': [1.0], 'GSM2411096': [1.0], 'GSM2411097': [1.0], 'GSM2411098': [1.0], 'GSM2411099': [1.0], 'GSM2411100': [1.0], 'GSM2411101': [1.0], 'GSM2411102': [1.0], 'GSM2411103': [1.0], 'GSM241110

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the data are in the format '11715100_at', '11715101_s_at', etc.
# These appear to be Affymetrix probe IDs, not human gene symbols.
# Affymetrix probe IDs need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key columns in the gene annotation dataframe
# 'ID' contains the Affymetrix probe IDs (like '11715100_at')
# 'Gene Symbol' contains the gene symbols (like 'HIST1H3G')
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the mapping dataframe using the function from the library
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 4. Preview the result to verify
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First 5 genes:")
print(gene_data.index[:5])


Gene expression data after mapping:
Shape: (19963, 63)
First 5 genes:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Clinical Data Extraction (which was skipped in Step 2)
# Use the geo_select_clinical_features function to extract clinical features
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Extracted clinical data with shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Deal with the gene mapping issue
print("\nChecking gene mapping results...")
is_gene_available = True

# Since mapping with Entrez IDs didn't work in Step 6, 
# we'll try to use the original gene IDs directly
if gene_data.shape[0] > 0:
    # Save the gene data with original identifiers
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    is_gene_available = False
    print("No valid gene expression data found.")

# 3. Link the clinical and genetic data
if is_gene_available:
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
    print(f"Linked data shape: {linked_data.shape}")

    # 4. Handle missing values in the linked data systematically
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After handling missing values, linked data shape: {linked_data.shape}")

    # 5. Determine whether the trait and demographic features are severely biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_biased}")
else:
    linked_data = pd.DataFrame()
    is_biased = True
    print("Cannot link data as gene expression data is not available.")

# 6. Conduct final quality validation and save cohort information
note = "SuperSeries with multiple disease conditions. Gene mapping approach using Entrez IDs was unsuccessful. The dataset includes obesity samples but may lack proper gene annotations."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Extracted clinical data with shape: (1, 63)
Clinical data preview:
{'GSM2411058': [1.0], 'GSM2411059': [1.0], 'GSM2411060': [1.0], 'GSM2411061': [1.0], 'GSM2411062': [1.0], 'GSM2411063': [1.0], 'GSM2411064': [1.0], 'GSM2411065': [1.0], 'GSM2411066': [1.0], 'GSM2411067': [1.0], 'GSM2411068': [1.0], 'GSM2411069': [1.0], 'GSM2411070': [1.0], 'GSM2411071': [1.0], 'GSM2411072': [1.0], 'GSM2411073': [1.0], 'GSM2411074': [1.0], 'GSM2411075': [1.0], 'GSM2411076': [1.0], 'GSM2411077': [1.0], 'GSM2411078': [1.0], 'GSM2411079': [1.0], 'GSM2411080': [1.0], 'GSM2411081': [1.0], 'GSM2411082': [1.0], 'GSM2411083': [1.0], 'GSM2411084': [1.0], 'GSM2411085': [1.0], 'GSM2411086': [1.0], 'GSM2411087': [0.0], 'GSM2411088': [0.0], 'GSM2411089': [0.0], 'GSM2411090': [0.0], 'GSM2411091': [0.0], 'GSM2411092': [1.0], 'GSM2411093': [1.0], 'GSM2411094': [1.0], 'GSM2411095': [1.0], 'GSM2411096': [1.0], 'GSM2411097': [1.0], 'GSM2411098': [1.0], 'GSM2411099': [1.0], 'GSM2411100': [1.0], 'GSM2411101': [1.0], 'GSM2411

Gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE90713.csv
Linked data shape: (63, 19964)


After handling missing values, linked data shape: (63, 19964)
For the feature 'Adrenocortical_Cancer', the least common label is '0.0' with 5 occurrences. This represents 7.94% of the dataset.
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is fine.

Is trait biased: False


Linked data saved to ../../output/preprocess/Adrenocortical_Cancer/GSE90713.csv
